In [17]:
#FIN 514 project 4
#group:ZHI JI, YUTAO CHEN, YULONG LIU
import numpy as np
import matplotlib.pyplot as plt
r = 0.013
sigmaI = 0.22
sigmaJ = 0.25
cor = 0.6
s01 = 172.22
s02 = 101.35

#uncomment to compute delta
#s01 = s01+1
#s02 = s02+1

print(s01, s02)
num_path = 1000000
price = np.zeros((2,13,num_path))
price[0,0] = s01
price[1,0] = s02
payoff = np.zeros((13,num_path))
# caculate date intervals
date = [0,94,185,276,367,458,550,642,731,823,915,1007,1096]
dt = np.diff(date)/365
dt

172.22 101.35


array([0.25753425, 0.24931507, 0.24931507, 0.24931507, 0.24931507,
       0.25205479, 0.25205479, 0.24383562, 0.25205479, 0.25205479,
       0.25205479, 0.24383562])

In [18]:
ini_p = [s01,s02]
for i in range(1,13):
    w1 = np.random.normal(0,1,num_path)
    #w2 = np.random.normal(0,1,num_path)
    w2 = cor*w1 + np.sqrt(1-cor**2)*np.random.normal(0,1,num_path)
    st1 = price[0,i-1]*np.exp( (r - 0.5*sigmaI**2)*dt[i - 1] + sigmaI*np.sqrt(dt[i - 1])*w1)
    st2 = price[1,i-1]*np.exp( (r - 0.5*sigmaJ**2)*dt[i - 1] + sigmaJ*np.sqrt(dt[i - 1])*w2)
    #st2 = price[1,i-1]*np.exp( (r - 0.5*sigmaJ**2)*(63/360) + sigmaJ*cor*np.sqrt(63/360)*w1 +\
    #    sigmaJ*np.sqrt(1-cor**2)*np.sqrt(63/360)*w2)
    #assume proportional dividends
    price[0,i] = st1 - st1*0.007511
    price[1,i] = st2 - st2*0.007239
#print(price[1,:,1000])
#plt.plot(range(0,13),price[0,:,:])
#plt.show()

In [19]:
redeem = np.zeros((num_path))    # 0 means not redeemed, 1 is redeemed
for i in range(1,13):
    for j in range(0,num_path):
        if (i < 4):  #before redemption dates, only pay coupon
        
            if(price[0,i,j] >= 0.75*s01) and (price[1,i,j] >= 0.75*s02):
                payoff[i,j] = 0.20875
                
        elif (i >= 4) and (i < 12) and (redeem[j] == 0) :   # now you can redeem,
        
            # redeem the security if both are above initial price
            if(price[0,i,j] >= s01) and (price[1,i,j] >= s02):
                payoff[i,j] = 10 + 0.20875
                redeem[j] = 1
            elif(price[0,i,j] >= 0.75*s01) and (price[1,i,j] >= 0.75*s02):
                payoff[i,j] = 0.20875
                
        # payoff at maturity            
        elif (redeem[j] == 0): 
            # both above downside threshold level:
            if(price[0,i,j] >= 0.75*s01) and (price[1,i,j] >= 0.75*s02):
                payoff[i,j] = 10 + 0.20875
            # one is under
            else: 
                #$10 x share performance factor of the worst performing stock
                
                payoff[i,j] = 10*np.minimum(price[0,i,j]/s01,price[1,i,j]/s02) 


            

In [20]:
avg_payoff =  np.mean(payoff,axis = 1)       
#print(payoff[0,5,1000:2000])
print(avg_payoff) 
value = 0
for i in range(1,13):
    value += avg_payoff[i]*np.exp(-r*i*dt[i - 1])
value         

[0.         0.2046608  0.18872816 0.17290596 2.97000745 0.69770846
 0.45968285 0.34183794 0.26571923 0.22185611 0.18960927 0.15928943
 3.4752672 ]


9.122961251139456

s01+1 :8.969688472279785 - 8.9643623186761
s02+1 : 8.969688472279785 - 8.971507863511

In [132]:
deltas1 = 8.969688472279785 - 8.9643623186761
deltas2 = 8.969688472279785 - 8.971507863511
print('ds1',deltas1,'ds2',deltas2)

ds1 0.0053261536036846024 ds2 -0.0018193912312156613


In [133]:
# path = np.random.random_integers(0,10000)
# print(path)
# print('IBM',price[0,:,path])
# print('JNJ',price[1,:,path])